## 학습시킨 모델들을 테스트 

In [ ]:
from PIL import Image
import torch

### 모델 불러오기

In [ ]:
def get_model_trained(PATH) : # PATH = '/home/ubuntu/CUAI_2021/Advanced_Minkyu_Kim/Scalp_model_parameters/'
    model_CNN = models.densenet161(pretrained = True, memory_efficient = True).to(device)

    new_classifier = nn.Sequential(
                nn.Linear(in_features=2208, out_features=6, bias=True)
                ).to(device)

    for m in new_classifier.modules():
        if isinstance(m, nn.Linear) :
            nn.init.kaiming_uniform_(m.weight)

    model_CNN.classifier = new_classifier.to(device) # 교체
    
    model_Diagnoser = scalp_state_diagnoser().to(device)
    
    # 학습된 가중치들 불러오기
    PATH_model_CNN = PATH + 'model_CNN_parameter.pt'
    PATH_model_Diagnoser = PATH + 'model_Diagnoser_parameter.pt'
    
    model_CNN.load_state_dict(torch.load(PATH_model_CNN))
    model_Diagnoser.load_state_dict(torch.load(PATH_model_Diagnoser))
    
    return model_CNN, model_Diagnoser

### 모발 이미지로 증상별 중증도를 휙득

In [ ]:
def get_value(img_path, model_CNN, model_Diagnoser, device) :
    
    state_str_list = ["미세각질", "피지과다", "모낭사이홍반", "모낭홍반농포", "비듬", "탈모", "양호"]
    
    scalp_state_diagnoser = model_Diagnoser[0]
    scalp_severity_diagnoser = model_Diagnoser[1]
    
    # 입력받은 이미지 경로에서 가져온 이미지를 전처리
    to_tensor = transforms.ToTensor()
    img = to_tensor(Image.open(img_path).convert('RGB'))
    img.resize_(3, 224, 224)
    img = torch.divide(img, 255.0) # 텐서로 변경 후 이미지 리사이징하고 각 채널을 0~1 사이의 값으로 만들어버림
    img = torch.unsqueeze(img, 0).to(device)
    
    # 전처리
    model_CNN.eval()
    model_Diagnoser.eval()
    
    output = model_CNN(img)
    output_state_per_class = model_Diagnoser(output)
    
    # 결과 출력
    print_str = "검사 결과, "
    for i in range(len(state_str_list)) : 
        print_str += state_str_list[i] + "이(가) %.4f 만큼, " % Test_Scalp_classifier_Dataset[809][1][i].item()

    print_str = print_str[:-2]
    
    print_str += " 있습니다."

    print(print_str)
    
    return output_state_per_class
    

##### 정확도 계산하는 함수. 혹시 몰라서 제작

In [ ]:
def calc_acc(Test_datasets, model_CNN, model_Diagnoser, device) :
    
    Test_Scalp_Health_Dataset = Test_datasets[0]
    Test_Scalp_classifier_Dataset = Test_datasets[1]
    
    
    for i in range(len(Test_Scalp_Health_Dataset)) :
        output = model_CNN(Test_Scalp_Health_Dataset[i][0])
        output_state = scalp_state_diagnoser(output)
        output_severity = scalp_severity_diagnoser(output)
        

### 모델 테스트 코드 예시

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_CNN, model_Diagnoser = get_model_trained('/home/ubuntu/CUAI_2021/Advanced_Minkyu_Kim/Scalp_model_parameters/')
output_state_per_class = get_value(img_path, model_CNN, model_Diagnoser, device)